# **Experimental Part**

В данной работе рассматриваются различные подходы к построению Retrieval-Augmented Generation (RAG), включая:

- **Trivial RAG** — базовый подход к RAG;
- **Trivial RAG + BM25** — улучшенный вариант с использованием алгоритма BM25 для более точного поиска релевантной информации;
- **Graph RAG** — подход, который сочетает использование базы знаний (knowledge base) с Trivial RAG.

Перед реализацией указанных подходов будет проведена аннотация отдельных чанков (фрагментов данных) с использованием вариации метода **Contextual Retrieval**, который позволит более точно извлекать и обогощать информацию из различных модулей кода проекта.

Для оценки качества реализованных подходов будет использован инструмент **RAGAS**, с помощью которого будут сформированы примитивные тесты, позволяющие объективно сравнить результаты работы различных методов.

Основные шаги, предусмотренные в ходе работы:

1. **Аннотация данных:**  
   - Разделение проекта на чанки;  
   - Аннотирование чанков с применением Contextual Retrieval.

2. **Реализация RAG-подходов:**  
   - Разработка и настройка Trivial RAG, Trivial RAG + BM25, Graph RAG.

3. **Оценка качества:**  
   - Создание тестов с использованием RAGAS;  
   - Сравнительный анализ результатов.

Результаты экспериментов помогут определить наиболее эффективный подход к построению RAG для решения поставленных задач.


## **Data Preprocessing and Annotation**

Данный подход наиболее полно рассмотрен в отдельной тетрадке под названием **data_prprocessing**, поэтому подробно на нём останавливаться не будем, а лишь перечислим статьи, в которых описаны использованные методы:

- [Sufficient Context: A New Lens on Retrieval Augmented Generation Systems](https://arxiv.org/abs/2411.06037)


In [1]:
# Тестовый запрос для проверки работоспособности
query = """
вот детальная спецификация Предоставленная информация и требования позволят составить детальную спецификацию задач для реализации функциональности отображения данных из формы обратной связи. В основе решения будет лежать применение технологии Java Spring Boot для бэкенда и использование базы данных PostgreSQL.
Спецификация задач Бэкенд-разработка API для получения списка обращений
Ендпоинт: GET /api/v1/feedback
Функциональность:
Возвращает список обращений. Поддерживает фильтрацию параметрами: type и priority. Поддерживает сортировку по параметру created_at, с возможностью указания порядка (возрастание или убывание). Реализация:
Использовать Spring Data JPA для управления запросами, включая Specification интерфейс для динамических запросов с фильтрацией и сортировкой. Конструировать SQL запросы с использованием Criteria API или JPQL. Возвращаемый ответ:
JSON-массив объектов, каждый из которых включает id, user_id, type, priority, description, и created_at. API для получения детальной информации об обращении
Ендпоинт: GET /api/v1/feedback/{{id}}
Функциональность:
Возвращает полную информацию об обращении, включая связанные файлы. Реализация:
Использовать JPA для получения данных из таблицы feedback. Использовать join-запросы или в запросе использовать feedback_documents для сопоставления с таблицей document. Возвращаемый ответ:
JSON-объект с подробными данными обращения и массив URL-адресов на прикрепленные файлы. Метод сохранения файла в Document:
Метод: public Document create(MultipartFile file, UserPrincipal principal, boolean useOpenAI) Изменения: Добавить флаг useOpenAI для определения необходимости загрузки файлов в OpenAI. Логика загрузки: String fileApiId = null; if (useOpenAI) { fileApiId = openaiClient.uploadFile(file); } String uploadedObjectName = minioService.uploadFile(file);
Document document = new Document(); document.setOriginalFileName(file.getOriginalFilename()); document.setObjectName(uploadedObjectName); document.setUser(user); document.setSize(file.getSize()); document.setFileApiId(fileApiId);
return documentRepository.save(document); Базы данных:
Миграции: Использовать инструмент миграции схемы базы данных, такой как Flyway или Liquibase, для создания и обновления таблиц feedback, document, и feedback_documents.
напиши мне код на Spring Boot для реализации. Учитывай что мы сейчас используем DTO для передачи данных.

"""

## **Trivial RAG**

Для гибкости в данном разделе не везде будем использовать встроенные методы/классы langchain, но во благо гибкости.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
import os
import tiktoken
from openai import OpenAI
from typing import List, Tuple
import dotenv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pprint import pprint

dotenv.load_dotenv()


После аннотирования отдельных фрагментов кода (которые в данном случае эквивалентны чанкам) с использованием подхода **Contextual Retrieval**, мы объединили их в единый документ с обогащенным контекстом. Далее документ будет повторно разделён на чанки, которые станут основой для обучения модели с применением следующих подходов: **Trivial RAG, RAG + BM25** и **Graph RAG**.

В соответствии с нашим аннотирования используем strategy chunking по символу (символьному слову).


In [ ]:
with open('backend.txt', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = CharacterTextSplitter(
    separator="Файл:",
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)

chunks = text_splitter.split_text(text)

# Фильтруем чанки, оставляя только те, что меньше 8192 токенов (бейзлайн обусловленный длинной входного контекста модели)
encoding = tiktoken.get_encoding("cl100k_base")
filtered_chunks = []
for chunk in chunks:
    num_tokens = len(encoding.encode(chunk))
    if num_tokens <= 8192:
        filtered_chunks.append(chunk)

print(f"Total {len(filtered_chunks)} chunks received after filtering")
for i, chunk in enumerate(filtered_chunks, 1):
    print(f"\nChunk {i}:")
    print(chunk.strip())
    print(f"Chunk size: {len(chunk)} characters")
    print(f"Number of tokens: {len(encoding.encode(chunk))}")


Также посмотрим на количество токенов в нашем документе, чтобы продемонстрировать что число токенов >100_000, что является верзней границей для чистого применения LLM и говорит об необходимости использования RAG.


In [4]:
def num_tokens_from_text(text: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [ ]:
num_tokens_from_text(text, "o200k_base")

Теперь выполним генерацию ембеддингов для каждого чанка, чтобы использовать в качестве векторов для RAG.



In [6]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def generate_embeddings(input_texts: List[str], model: str = "text-embedding-ada-002") -> List[List[float]]:
    """Генерация эмбеддингов с помощью OpenAI API.
    
    Args:
        input_texts: список текстов (чанки) для генерации эмбеддингов
        model: модель для генерации эмбеддингов (по умолчанию text-embedding-ada-002), взяли просто проприетарную
        
    Returns:
        List[List[float]]: список векторов эмбеддингов для каждого входного текста
    """
    response = client.embeddings.create(
        input=input_texts,
        model=model
    )
    return [data.embedding for data in response.data]


In [ ]:
embeddings = generate_embeddings(filtered_chunks)

print(f"A total of {len(embeddings)} embedding vectors have been generated")


Я теперь реализуем простой vector retrieval для нашего документа, который будет использоваться для trivial RAG.

In [8]:
def vector_retrieval(query: str, top_k: int = 5) -> List[int]:
    """
    Получение top-k наиболее релевантных документов на основе косинусного сходства (бейзлайн, остальные метрики сходства аргументировать сложнее) между
    эмбеддингом запроса и эмбеддингами документов.
    
    Args:
        query (str): Текст запроса
        top_k (int): Количество документов для получения (по умолчанию 5)
        
    Returns:
        List[int]: Список индексов top-k наиболее релевантных документов
    """
    query_embedding = generate_embeddings([query])[0]
    embeddings_array = np.array(embeddings)
    
    # Вычисляем косинусное сходство между запросом и всеми документами
    similarities = cosine_similarity([query_embedding], embeddings_array)[0]
    
    # Получаем индексы top-k документов с наибольшим сходством
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    return list(top_indices)


In [ ]:
vector_retrieval(query, 5)

С учетом всего вышеуказанного кода (плохая практика, что он в глобальном пространстве), мы сформируем класс **TrivialRAG** для удобства обращения и рпдеоставления единого интерфейса.

In [10]:
from src.prompt_templates import simple_rag_prompt

class SimpleRAG:
    def __init__(self, embeddings: List[np.ndarray], chunks: List[str], client: OpenAI, prompt_template: str = simple_rag_prompt):
        """
        Инициализация класса SimpleRAG.

        Args:
            embeddings (List[np.ndarray]): Эмбеддинги документов
            chunks (List[str]): Исходные текстовые чанки
            client (OpenAI): клиент OpenAI
            prompt_template (str): Шаблон промпта (по умолчанию загружается из prompt_templates)
        """
        self.embeddings = embeddings
        self.chunks = chunks
        self.client = client
        self.prompt_template = prompt_template
        
    def _get_relevant_chunks(self, query: str, top_k: int = 5) -> List[str]:
        """
        Получение релевантных чанков на основе косинусного сходства.

        Args:
            query (str): Текст запроса
            top_k (int): Количество чанков для получения

        Returns:
            List[str]: Список релевантных чанков
        """
        query_embedding = generate_embeddings([query])[0]
        embeddings_array = np.array(self.embeddings)
        similarities = cosine_similarity([query_embedding], embeddings_array)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [self.chunks[i] for i in top_indices]

    def retrieve_and_generate(self, query: str, top_k: int = 5) -> Tuple[List[str], str]:
        """
        Получение релевантных чанков и генерация ответа.

        Args:
            query (str): Запрос пользователя
            top_k (int): Количество релевантных чанков

        Returns:
            Tuple[List[str], str]: Кортеж из списка использованных чанков и сгенерированного ответа
        """
        relevant_chunks = self._get_relevant_chunks(query, top_k)
        context = "\n\n".join(relevant_chunks)
        
        prompt = self.prompt_template.format(
            context=context,
            query=query
        )
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2
        )
        
        return relevant_chunks, response.choices[0].message.content

    def retrieve(self, query: str, top_k: int = 5) -> Tuple[List[int], str]:
        """
        Основной метод для получения ответа на запрос.
        
        Args:
            query (str): Запрос пользователя
            top_k (int): Количество релевантных чанков
            
        Returns:
            Tuple[List[int], str]: Кортеж из индексов использованных чанков и сгенерированного ответа
        """
        relevant_chunks, answer = self.retrieve_and_generate(query, top_k)
        chunk_indices = [self.chunks.index(chunk) for chunk in relevant_chunks]
        return answer, chunk_indices

In [11]:
simple_rag = SimpleRAG(embeddings=embeddings, chunks=chunks, client=client)  
simple_response, _ = simple_rag.retrieve(query=query)

In [ ]:
pprint(simple_response)

## **Trivial RAG + BM25**

Комбинированный подход, сочетающий поиск по эмбеддингам и статистический метод **BM25**, представляет собой более расширенную практику, которая демонстрирует высокую эффективность. Данный метод был подробно описан в [статье Anthropic, опубликованной в сентябре 2024 года](https://www.anthropic.com/news/contextual-retrieval). 

В следующем разделе мы реализуем данный подход на практике.


In [ ]:
import bm25s
from collections import defaultdict
from sentence_transformers import CrossEncoder

In [ ]:
retriever = bm25s.BM25(corpus=filtered_chunks)
retriever.index(bm25s.tokenize(filtered_chunks))
results, scores = retriever.retrieve(bm25s.tokenize(query), k=5)

In [15]:
def bm25_retrieval(query: str, top_k: int = 5) -> List[int]:
    """
    Получение top-k наиболее релевантных документов с помощью алгоритма BM25.
    
    Args:
        query (str): Текст запроса
        top_k (int): Количество документов для получения (по умолчанию 5)
        
    Returns:
        List[int]: Список индексов top-k наиболее релевантных документов
    """
    # Получаем результаты и оценки релевантности с помощью BM25
    results, _ = retriever.retrieve(bm25s.tokenize(query), k=top_k)
    
    # Преобразуем результаты в индексы исходных документов
    return [filtered_chunks.index(doc) for doc in results[0]]


In [ ]:
bm25_retrieval(query , top_k = 5)

Добавим вычисление reciprocal rank fusion для нашего документа, который будет использоваться для полуения совместных результатов с bm25 и embedding retrieval.

In [17]:
def reciprocal_rank_fusion(*list_of_list_ranks_system, K=60):
    """
    Объединение результатов ранжирования от нескольких IR систем с помощью Reciprocal Rank Fusion.

    Args:
    list_of_list_ranks_system: Ранжированные результаты от разных IR систем.
    K (int): Константа, используемая в формуле RRF (по умолчанию 60, просто часто используют 60).

    Returns:
    Кортеж из списка отсортированных документов по оценке и отсортированных документов
    """
    # Словарь для хранения RRF оценок
    rrf_map = defaultdict(float)

    # Вычисляем RRF оценку для каждого результата в каждом списке
    for rank_list in list_of_list_ranks_system:
        for rank, item in enumerate(rank_list, 1):
            rrf_map[item] += 1 / (rank + K)

    # Сортируем элементы по их RRF оценкам в порядке убывания
    sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

    # Возвращаем кортеж из списка отсортированных документов по оценке и отсортированных документов
    return sorted_items, [item for item, score in sorted_items]


In [ ]:
vector_top_k = vector_retrieval(query, top_k=5)
bm25_top_k = bm25_retrieval(query, top_k=5)

hybrid_top_k = reciprocal_rank_fusion(vector_top_k, bm25_top_k)
hybrid_top_k[1]

In [ ]:
for index in hybrid_top_k[1]:
  print(f"Chunk Index {index} : {filtered_chunks[index]}")

Теперь мы улучшим качество результатов, применяя реранкинг с учетом окружающего контекста. Это позволит нам более точно оценить релевантность документов, основываясь на их содержании и связи с запросом. Реранкинг поможет выделить наиболее подходящие документы, учитывая не только их первоначальные оценки, но и контекст, в котором они были найдены. Воспользуемся легким с hf и самым используемым.

In [ ]:
documents = [filtered_chunks[idx] for idx in hybrid_top_k[1]]
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
pairs = [[query, doc] for doc in documents]
scores = cross_encoder.predict(pairs)
reranked_results = list(zip(documents, scores))
reranked_results.sort(key=lambda x: x[1], reverse=True)
print("Top-3 results after reranking:\n")
for doc, score in reranked_results[:3]:
    print(f"Document: {doc}")
    print(f"Relevance score: {score}\n")


In [ ]:
retreived_chunks = ''

for doc, score in reranked_results:
    retreived_chunks += doc + '\n\n'

print(retreived_chunks)


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=[
        {"role": "user", "content": f"Answer the question: {query}. Here is relevant information: {retreived_chunks} Response only in Russian. It is necessary to implement everything in detail in the code. You are a coding assistant."},
    ],
)

print(response.choices[0].message.content)


In [23]:
from src.prompt_templates import bm25_rag_prompt

class BM25RAG:
    def __init__(self, embeddings: List[np.ndarray], chunks: List[str], client: OpenAI, prompt_template: str = bm25_rag_prompt):
        """
        Инициализация класса BM25RAG.

        Args:
            embeddings (List[np.ndarray]): Эмбеддинги документов
            chunks (List[str]): Исходные текстовые чанки
            client (OpenAI): клиент OpenAI
            prompt_template (str): Шаблон промпта
        """
        self.embeddings = embeddings
        self.chunks = chunks
        self.client = client
        self.prompt_template = prompt_template
        self.bm25_retriever = bm25s.BM25(corpus=chunks)
        self.bm25_retriever.index(bm25s.tokenize(chunks))
        self.cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
    def _vector_retrieval(self, query: str, top_k: int = 5) -> List[int]:
        """
        Получение top-k наиболее релевантных документов на основе векторного поиска.
        """
        query_embedding = generate_embeddings([query])[0]
        embeddings_array = np.array(self.embeddings)
        similarities = cosine_similarity([query_embedding], embeddings_array)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return list(top_indices)
    
    def _bm25_retrieval(self, query: str, top_k: int = 5) -> List[int]:
        """
        Получение top-k наиболее релевантных документов с помощью BM25.
        """
        results, _ = self.bm25_retriever.retrieve(bm25s.tokenize(query), k=top_k)
        return [self.chunks.index(doc) for doc in results[0]]
    
    def _rerank_results(self, query: str, documents: List[str], top_k: int = 3) -> List[str]:
        """
        Реранкинг результатов с использованием кросс-энкодера.
        """
        pairs = [[query, doc] for doc in documents]
        scores = self.cross_encoder.predict(pairs)
        reranked_results = list(zip(documents, scores))
        reranked_results.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in reranked_results[:top_k]]

    def retrieve_and_generate(self, query: str, top_k: int = 5) -> Tuple[List[str], str]:
        """
        Получение релевантных чанков и генерация ответа.

        Args:
            query (str): Запрос пользователя
            top_k (int): Количество релевантных чанков

        Returns:
            Tuple[List[str], str]: Кортеж из списка использованных чанков и сгенерированного ответа
        """
        # Получаем результаты от обоих методов
        vector_top_k = self._vector_retrieval(query, top_k)
        bm25_top_k = self._bm25_retrieval(query, top_k)
        
        # Объединяем результаты с помощью RRF
        _, hybrid_indices = reciprocal_rank_fusion(vector_top_k, bm25_top_k)
        
        # Получаем документы для реранкинга
        documents = [self.chunks[idx] for idx in hybrid_indices]
        
        # Выполняем реранкинг
        reranked_chunks = self._rerank_results(query, documents)
        
        # Объединяем контекст
        context = "\n\n".join(reranked_chunks)
        
        # Генерируем ответ
        prompt = self.prompt_template.format(
            context=context,
            query=query
        )
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2
        )
        
        return reranked_chunks, response.choices[0].message.content

    def retrieve(self, query: str, top_k: int = 5) -> Tuple[str, List[int]]:
        """
        Основной метод для получения ответа на запрос.
        
        Args:
            query (str): Запрос пользователя
            top_k (int): Количество релевантных чанков
            
        Returns:
            Tuple[str, List[int]]: Кортеж из сгенерированного ответа и индексов использованных чанков
        """
        relevant_chunks, answer = self.retrieve_and_generate(query, top_k)
        chunk_indices = [self.chunks.index(chunk) for chunk in relevant_chunks]
        return answer, chunk_indices

In [ ]:
bm25_rag = BM25RAG(embeddings=embeddings, chunks=chunks, client=client)  
bm25_response, _ = bm25_rag.retrieve(query=query)

In [ ]:
pprint(bm25_response)

<font size="5">Построение Knowledge graph</font>

На основе текущих возможностей фремворка langchain построим наш knowledge граф знаний на основе текущего репозитория.

In [26]:
import os
import time
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.schema import Document
from langchain.callbacks import get_openai_callback

class KnowledgeBaseORM:
    def __init__(self):
        self.graph = Neo4jGraph(
            url=os.getenv("NEO4J_URL"),
            username=os.getenv("NEO4J_USERNAME", "neo4j"),
            password=os.getenv("NEO4J_PASSWORD"),
        )
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
        self.llm_transformer = LLMGraphTransformer(
            llm=self.llm,
            allowed_nodes=[
                "Class", "Method", "Function", "Interface",
                "Controller", "Service", "Repository", "Entity", "DTO",
                "Variable", "Parameter", "ReturnType"
            ],
            allowed_relationships=[
                ("Class", "IMPLEMENTS", "Interface"),
                ("Class", "EXTENDS", "Class"),
                ("Method", "BELONGS_TO", "Class"),
                ("Method", "RETURNS", "ReturnType"),
                ("Method", "ACCEPTS", "Parameter"),
                ("Controller", "USES", "Service"),
                ("Service", "USES", "Repository"),
                ("Repository", "MANAGES", "Entity"),
                ("Controller", "RETURNS", "DTO"),
                ("Entity", "MAPS_TO", "DTO"),
                ("Service", "TRANSFORMS", "DTO"),
                ("Function", "CALLS", "Function"),
                ("Method", "CALLS", "Method"),
                ("Variable", "TYPE_OF", "Class")
            ],
            node_properties=True,
        )
        self.total_cost = 0
        self.total_tokens = 0
        self.timeout = 180

    def load_documents(self, chunks: list):
        """
        Load documents from a list of chunks and convert them into graph documents.
        Analyzes the code for:
        - Classes and their hierarchy
        - Methods and their signatures
        - Used data types
        - Relationships between components
        - Architectural patterns

        Args:
            chunks (list): List of text chunks to load.
        """
        for idx, chunk in enumerate(chunks, 1):
            start_time = time.time()
            try:
                with get_openai_callback() as cb:
                    if isinstance(chunk, tuple):
                        document = Document(page_content=chunk[0])
                    else:
                        document = Document(page_content=chunk)
                    
                    if time.time() - start_time > 180:  # Increased timeout to 3 minutes
                        print(f"Chunk {idx} skipped due to exceeding processing time (>180 sec)")
                        continue
                        
                    graph_documents = self.llm_transformer.convert_to_graph_documents([document])
                    self.graph.add_graph_documents(graph_documents)
                    
                    print(f"Chunk {idx} successfully analyzed and added to the knowledge graph.")
                    print(f"Cost of analyzing chunk {idx}: ${cb.total_cost:.4f}")
                    print(f"Tokens used: {cb.total_tokens}")
                    print(f"Analysis time: {time.time() - start_time:.2f} sec\n")
                    
                    self.total_cost += cb.total_cost
                    self.total_tokens += cb.total_tokens
                    
                    # Adding a 5-second delay between requests
                    time.sleep(5)
                    
            except Exception as e:
                print(f"Error analyzing chunk {idx}: {str(e)}")
                continue

    def clear_graph(self):
        """
        Clear the existing knowledge graph and prepare for new analysis.
        """
        self.graph.query("MATCH (n) DETACH DELETE n")
        print("Knowledge graph cleared for new analysis.")

    def build_knowledge_graph(self, filtered_chunks: list):
        """
        Build a knowledge graph from the codebase.
        Analyzes the structure of the code and creates relationships between components.

        Args:
            filtered_chunks (list): List of filtered code chunks.
        """
        self.clear_graph()
        self.load_documents(filtered_chunks)
        print("\nFinal analysis statistics:")
        print(f"Total analysis cost: ${self.total_cost:.4f}")
        print(f"Total number of processed tokens: {self.total_tokens}")
        print("Knowledge graph of the codebase successfully built.")

In [27]:
# orm = KnowledgeBaseORM()
# orm.build_knowledge_graph(filtered_chunks)

In [35]:
from langchain_openai import ChatOpenAI
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.graphs.neo4j_graph import Neo4jGraph
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from typing import List, Tuple
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from src.prompt_templates import graph_rag_prompt

class GraphRAG:
    def __init__(self, embeddings: List[np.ndarray], chunks: List[str], client: OpenAI, prompt_template: str = graph_rag_prompt):
        """
        Инициализация класса GraphRAG.

        Args:
            embeddings (List[np.ndarray]): Эмбеддинги документов
            chunks (List[str]): Исходные текстовые чанки
            client (OpenAI): клиент OpenAI
            prompt_template (str): Шаблон промпта
        """
        self.embeddings = embeddings
        self.chunks = chunks
        self.client = client
        self.prompt_template = prompt_template
        
        # Создаем LLM из langchain
        self.llm = ChatOpenAI(
            temperature=0.2,
            model_name="gpt-4o",
            openai_api_key=os.getenv('OPENAI_API_KEY')
        )
        
        # Подключение к Neo4j
        self.graph = Neo4jGraph(
            url=os.getenv("NEO4J_URL"),
            username=os.getenv("NEO4J_USERNAME", "neo4j"),
            password=os.getenv("NEO4J_PASSWORD"),
            database=os.getenv("NEO4J_DATABASE", "neo4j")
        )
        
        # Инициализация цепочек запросов с langchain LLM
        self.class_chain = GraphCypherQAChain.from_llm(
            llm=self.llm,
            graph=self.graph,
            verbose=True,
            allow_dangerous_requests=True
        )
        
        self.method_chain = GraphCypherQAChain.from_llm(
            llm=self.llm,
            graph=self.graph,
            verbose=True,
            allow_dangerous_requests=True
        )
        
        # Промпты для объединения контекстов и генерации кода
        self.combine_prompt = PromptTemplate(
            input_variables=["class_context", "method_context", "user_question"],
            template=(
                "Контекст по классам и интерфейсам:\n{class_context}\n\n"
                "Контекст по методам:\n{method_context}\n\n"
                "Пользовательский запрос:\n{user_question}\n\n"
                "Сформируй итоговое описание, которое объединяет оба результата."
            )
        )
        
        self.code_prompt = PromptTemplate(
            input_variables=["combined_context", "user_question"],
            template=(
                "Ниже приведён агрегированный контекст, полученный из базы знаний Neo4j:\n"
                "{combined_context}\n\n"
                "Пользовательский запрос:\n"
                "{user_question}\n\n"
                "На основе приведённого контекста сгенерируй код на Spring Boot для реализации API."
            )
        )
        
        self.combine_chain = LLMChain(
            llm=self.llm,
            prompt=self.combine_prompt,
            verbose=True
        )
        
        self.code_chain = LLMChain(
            llm=self.llm,
            prompt=self.code_prompt,
            verbose=True
        )

    def _vector_retrieval(self, query: str, top_k: int = 5) -> List[int]:
        """
        Получение top-k наиболее релевантных документов на основе векторного поиска.
        """
        query_embedding = generate_embeddings([query])[0]
        embeddings_array = np.array(self.embeddings)
        similarities = cosine_similarity([query_embedding], embeddings_array)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return list(top_indices)

    def retrieve_and_generate(self, query: str) -> Tuple[List[str], str]:
        """
        Получение контекста из графа знаний и генерация ответа.

        Args:
            query (str): Запрос пользователя

        Returns:
            Tuple[List[str], str]: Кортеж из списка использованных контекстов и сгенерированного ответа
        """
        for attempt in range(10):
            try:
                # Получаем контекст по классам и интерфейсам
                class_context = self.class_chain.run(
                    "Какие классы и интерфейсы задействованы? " + query
                )

                # Получаем контекст по методам
                method_context = self.method_chain.run(
                    "Какие методы используются в данных классах? " + query
                )

                # Получаем результаты векторного поиска
                vector_top_k = self._vector_retrieval(query)
                vector_context = [self.chunks[idx] for idx in vector_top_k]

                # Объединяем контексты
                combined_context = self.combine_chain.run(
                    class_context=class_context,
                    method_context=method_context,
                    user_question=query
                )

                # Добавляем контекст из векторного поиска
                combined_context += "\n\n" + "\n".join(vector_context)

                # Генерируем ответ
                response = self.code_chain.run(
                    combined_context=combined_context,
                    user_question=query
                )

                contexts = [class_context, method_context, combined_context]
                return contexts, response
            
            except Exception as e:
                if attempt == 9:  # Если это последняя попытка
                    return [], ""  # Возвращаем пустой контекст
                continue  # Пытаемся снова

    def retrieve(self, query: str) -> Tuple[str, List[str]]:
        """
        Основной метод для получения ответа на запрос.
        
        Args:
            query (str): Запрос пользователя
            
        Returns:
            Tuple[str, List[str]]: Кортеж из сгенерированного ответа и использованных контекстов
        """
        contexts, answer = self.retrieve_and_generate(query)
        return answer, contexts

In [ ]:
graph_rag = GraphRAG(embeddings=embeddings, chunks=filtered_chunks, client=client)
response, _ = graph_rag.retrieve(query=query)

In [ ]:
pprint(response)

<font size="5">Ragas</font>

In [55]:
from typing import List
import random

def generate_test_cases(chunks: List[str], n_samples: int = 30) -> List[dict]:
    """
    Генерация тестовых примеров с помощью LLM
    """
    test_cases = []
    
    for _ in range(n_samples):
        # Случайно выбираем чанк
        chunk = random.choice(chunks)
        
        # Генерируем вопрос на основе чанка
        prompt = f"""На основе следующего фрагмента кода сгенерируй технический вопрос:
        {chunk}
        
        Вопрос должен быть связан с реализацией конкретной функциональности.
        """
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        
        question = response.choices[0].message.content
        
        # Генерируем эталонный ответ на основе контекста
        reference_prompt = f"""На основе следующего фрагмента кода дай подробный ответ на вопрос:
        
        Контекст: {chunk}
        
        Вопрос: {question}
        """
        
        reference_response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[{"role": "user", "content": reference_prompt}]
        )
        
        reference_answer = reference_response.choices[0].message.content
        
        test_cases.append({
            "question": question,
            "context": chunk,
            "reference_answer": reference_answer
        })
    
    return test_cases

def evaluate_rag_system(test_cases: List[dict], rag_system) -> dict:
    """
    Формирование датасета RAG системы 
    """
    results = []
    
    for case in test_cases:
        # Добавляем обработку ошибок с 5 попытками
        max_attempts = 5
        attempt = 0
        answer = None
        
        while attempt < max_attempts:
            try:
                answer = rag_system.retrieve(case["question"])
                break
            except Exception:
                attempt += 1
                if attempt == max_attempts:
                    print(f"Пропускаем вопрос после {max_attempts} неудачных попыток: {case['question']}")
                    continue
        
        if answer is not None:
            results.append({
                "question": case["question"],
                "context": case["context"],
                "answer": answer,
                "reference_answer": case["reference_answer"]
            })
    
    return {"samples": results}

# # Генерируем тестовые примеры
# test_cases = generate_test_cases(filtered_chunks)

# #Оцениваем Simple RAG
# simple_rag_dataset = evaluate_rag_system(test_cases, SimpleRAG(embeddings=embeddings, chunks=filtered_chunks, client=client))

# #Оцениваем Graph RAG
# graph_rag_dataset = evaluate_rag_system(test_cases, GraphRAG(embeddings=embeddings, chunks=filtered_chunks, client=client))

# #Оцениваем RAG + BM25
# bm25_rag_dataset = evaluate_rag_system(test_cases, BM25RAG(embeddings=embeddings, chunks=filtered_chunks, client=client))


In [ ]:
print("\nSimple RAG results:")
pprint(simple_rag_dataset)
print("Graph RAG results:")
pprint(graph_rag_dataset)
print("\nRAG + BM25 results:")
pprint(bm25_rag_dataset)


In [ ]:
from typing import List
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
)

def prepare_dataset(
    test_cases: List[dict],  
    system_answers: List[str]
) -> EvaluationDataset:
    """
    Подготовка датасета для ragas.
    test_cases: список словарей с ключами 'question', 'context', 'reference_answer'
    system_answers: список ответов от RAG (по одному ответу на каждый test_case)
    """
    samples = []
    for case, answer in zip(test_cases, system_answers):
        # Убедимся, что ответ является строкой
        if not isinstance(answer, str):
            answer = str(answer)

        # reference – это эталонный ответ
        # actual_output (или output) – это фактический ответ системы
        # question/context – входная пара
        samples.append(
            SingleTurnSample(
                question=case["question"],
                context=case["context"],
                actual_output=answer,             # фактический ответ
                user_input=case["question"],
                response=answer,                 # дублируем для удобства
                retrieved_contexts=[case["context"]],
                reference=case["reference_answer"]  # эталонный ответ
            )
        )
    return EvaluationDataset(samples=samples)

# Извлекаем ответы из датасетов систем
graph_rag_answers  = [sample["answer"] for sample in graph_rag_dataset["samples"]]
bm25_rag_answers   = [sample["answer"] for sample in bm25_rag_dataset["samples"]]
simple_rag_answers = [sample["answer"] for sample in simple_rag_dataset["samples"]]

# Формируем EvaluationDataset для каждой системы
simple_rag_data = prepare_dataset(test_cases, simple_rag_answers)
graph_rag_data  = prepare_dataset(test_cases, graph_rag_answers)
bm25_rag_data   = prepare_dataset(test_cases, bm25_rag_answers)

# Запускаем оценку по метрикам
metrics = [faithfulness, answer_relevancy, context_recall]

simple_rag_results = evaluate(simple_rag_data, metrics)
graph_rag_results  = evaluate(graph_rag_data, metrics)
bm25_rag_results   = evaluate(bm25_rag_data, metrics)

print("\nSimple RAG Results:")
for metric in metrics:
    metric_name = metric.name
    print(f"{metric_name}: {simple_rag_results[metric_name][0]:.2f}")

print("\nGraph RAG Results:")
for metric in metrics:
    metric_name = metric.name
    print(f"{metric_name}: {graph_rag_results[metric_name][0]:.2f}")

print("\nBM25 RAG Results:")
for metric in metrics:
    metric_name = metric.name
    print(f"{metric_name}: {bm25_rag_results[metric_name][0]:.2f}")


# Сравнение результатов – определяем, у какой из систем максимальный скор
print("\nComparison:")
for metric in metrics:
    metric_name   = metric.name
    simple_score  = simple_rag_results[metric_name][0]
    graph_score   = graph_rag_results[metric_name][0]
    bm25_score    = bm25_rag_results[metric_name][0]

    scores = {
        "Simple RAG": simple_score,
        "Graph RAG":  graph_score,
        "BM25 RAG":   bm25_score
    }
    best_system = max(scores, key=scores.get)

    print(
        f"{metric_name}: {best_system} performs better | "
        f"Simple RAG: {simple_score:.2f}, "
        f"Graph RAG: {graph_score:.2f}, "
        f"BM25 RAG: {bm25_score:.2f}"
    )
